In [ ]:
# Install packages
!pip install --upgrade openai==1.1.0 # Need to use 1.1.0 version only for compatibility!!
!pip install langchain langchain-experimental
!pip install psycopg2-binary
!pip install streamlit
!pip install pandas
!pip install sqlalchemy

In [8]:
# After installing langchain, you need to run 'openai migrate' command in cli for the first time.
import random
import psycopg2
import streamlit as st
import os
import openai
import psycopg2
import streamlit as st
import pandas as pd
import json
import glob
import openai
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.utilities import SQLDatabase
from langchain.prompts import ChatPromptTemplate
from langchain.cache import SQLAlchemyCache
from langchain.globals import set_llm_cache
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chains import LLMChain # These are not working on openai>=1.2.0
from langchain_experimental.sql import SQLDatabaseChain # These are not working on openai>=1.2.0

In [12]:
# Define some functions

def init_connection():
    """
    Initialize a SQL database connection using the credentials stored in st.secrets.
    Returns:
    connection (psycopg2.connection): The SQL database connection.
    """
	# the contents of st.secrets are in .streamlit/secrets.toml
    connection = psycopg2.connect(
        user = st.secrets['username'],
		password = st.secrets['password'],
		host = st.secrets['host'],
		port = st.secrets['port'],
		database = st.secrets['database']
	)
    return connection
     

def init_db(include_tables = []):
    """
    Initialize a SQL database db object for langchain using the credentials stored in st.secrets.

    Returns:
    db (SQLDatabase): The SQL database object.
    pg_uri (str): The PostgreSQL URI used for the connection.
    """

    # Retrieve the credentials from st.secrets
    user = st.secrets['username']
    password = st.secrets['password']
    host = st.secrets['host']
    port = st.secrets['port']
    database = st.secrets['database']
    
    # Create the PostgreSQL URI
    pg_uri = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
    
    # Create the SQLDatabase object
    if include_tables == []:    
        db = SQLDatabase.from_uri(pg_uri)
    else:
        db = SQLDatabase.from_uri(pg_uri, include_tables=include_tables,sample_rows_in_table_info=2)
    
    return db, pg_uri

### Task 1. Raw Date Processing and insert into SQL DB

In [5]:
# Connect to the SQL database and get the knowledge and sub_unit tables
connection = init_connection()
# create a table in the database
cursor = connection.cursor()
# load data from sql table 'knowledge_map_db.knowledge'.
query = "SELECT knowledge_id, knowledge_name FROM knowledge_map_db.knowledge;"
cursor.execute(query)
knowledge_table = 'knowledge_map_db.knowledge'
knowledge_columns = [description[0] for description in cursor.description]
knowledge_data = cursor.fetchall()

query = "SELECT sub_unit_id, sub_unit_name FROM knowledge_map_db.sub_unit;"
cursor.execute(query)
sub_unit_table = 'knowledge_map_db.sub_unit'
sub_unit_data = cursor.fetchall()
sub_unit_columns = [description[0] for description in cursor.description]
knowledge_str = f"{knowledge_table}, {knowledge_columns}, {knowledge_data}"
sub_unit_str = f"{sub_unit_table}, {sub_unit_columns}, {sub_unit_data}"# modify print as var
print(knowledge_str)
print(sub_unit_str)

knowledge_map_db.knowledge, ['knowledge_id', 'knowledge_name'], [(1, '다항식 관련 용어'), (2, '다항식의 사칙연산'), (3, '곱셈공식'), (4, '곱셈공식의 변형'), (5, '항등식의 정의'), (6, '항등식의 성질'), (7, '미정계수법'), (8, '나머지정리'), (9, '인수정리'), (10, '조립제법'), (11, '공통인수 묶기'), (12, '이차식의 인수분해 공식'), (13, '삼차식의 인수분해 공식'), (14, '치환에 의한 인수분해'), (15, '복잡한 식의 인수분해'), (16, '고차식의 인수분해'), (17, '복소수의 정의'), (18, '켤레복소수'), (19, '복소수의 사칙연산'), (20, '복소수가 서로 같을 조건'), (21, '음수의 제곱근'), (22, '켤레복소수의 성질'), (23, '허수단위의 순환성'), (24, '이차방정식과 이차함수의 관계'), (25, '이차함수의 그래프와 직선의 위치 관계'), (26, '이차방정식의 실근의 부호 판단'), (27, '이차방정식의 실근의 위치에 따른 조건'), (28, '이차함수의 최댓값'), (29, '이차함수의 최솟값'), (30, '삼차방정식과 사차방정식의 풀이'), (31, '삼차방정식의 근과 계수의 관계'), (32, '세 근을 알 때 삼차방정식 구하는 방법'), (33, '삼차방정식의 켤레근'), (34, '방정식의 허근의 성질'), (35, '해가 없거나 무수히 많은 연립일차방정식'), (36, '미지수가 2개인 연립이차방정식의 풀이'), (37, '대칭연립방정식'), (38, '공통근'), (39, '부정방정식의 풀이'), (40, '부등식 ax>b의 풀이 방법'), (41, '연립부등식의 풀이 방법'), (42, '절댓값 기호를 포함한 부등식의 풀이 방법'), (43, '이차부등식의 풀이 방법'), (44, '이차부등식이 항상 성립할 조건'), (45, '연립이차부등식의 풀이 방법'

In [13]:
# Define the templates for raw data processing prompts

templates = {}

templates["give problem"] = """
You are a high school mathematics instructor. Evaluate if every mathematical question in the MATH dataset is relevant to a topic commonly included in the high school curriculum and aligns with the level of high school education.

Details:
- Exemplary data format includes:
  - Problem: 'We roll a fair 6-sided die 5 times. What is the probability that we get a 6 in at most 2 of the rolls?'
  - Level: 'Level 5'
  - Type: 'Counting & Probability'
  - Solution: [Complex mathematical formula]

This is given problem
{problem}

Assess whether this math question is relevant to a topic commonly included in the high school curriculum and aligns with the level of high school education. Answer yes as 1 or no as 0.
"""

templates["knowledge_mapping"] = """
You are a high school mathematics instructor.
We need to map the given math question to the knowledge_map_db.knowledge table.

Exemplary data format includes:
Problem: "We roll a fair 6-sided die 5 times. What is the probability that we get a 6 in at most 2 of the rolls?"
Level: "Level 5"
Type: "Counting & Probability"
Solution: [Complex mathematical formula]

You need to identify 1-3 knowledge areas within the knowledge_map_db.knowledge table that are related to each question.
Table content is as follows:
{knowledge_str}

Simply print the ('knowledge_id', 'knowledge_name') in json format for each given problem.
""".format(knowledge_str=knowledge_str)


templates["unit_mapping"] = """
You are a high school mathematics instructor.
We need to map each given problem to the knowledge_map_db.sub_unit table.

Exemplary data format is as follows:
Problem: "We roll a fair 6-sided die 5 times. What is the probability that we get a 6 in at most 2 of the rolls?"
Level: "Level 5"
Type: "Counting & Probability"
Solution: [Complex mathematical formula]

You need to identify 1 unit within knowledge_map_db.sub_unit table that is related to each question.
Table content is as follows:
{sub_unit_str}  

Simply print the ('sub_unit_id', 'sub_unit_name') in json format for each given problem.
""".format(sub_unit_str=sub_unit_str)

templates["postprocess"] = """
You are a high school mathematics instructor. We will be processing data from the MATH dataset.

Exemplary data format is as follows:
Problem: "We roll a fair 6-sided die 5 times. What is the probability that we get a 6 in at most 2 of the rolls?"
Level: "Level 5"
Type: "Counting & Probability"
Solution: [Complex mathematical formula]

Expected data format for the knowledge_map_db.problem db is as follows:
question: text (Translate the original problem into Korean and encapsulate LaTeX expressions with $)
solution: text (Translate the original solution into Korean and encapsulate LaTeX expressions with $)
hint: text (Create 1 hint providing a concise sentence that emphasizes the key concept or guides problem-solving
level: int (Use the original level.)
step_criteria: json (Create 2-5 steps based on the solution. Format: '{"criteria1": …, "criteria2": …, …}'
step_score: 'int[]' (Assign scores to each step for 'step_criteria', totaling 10.)
competence: 'numeric[]' (Create a relevance score vector for five competence elements: 'problem-solving', 'information processing', 'reasoning', 'communication', 'connection', according to the Korean mathematics curriculum and your own estimation. The total should sum up to 1.)

Simply print the ('question', 'solution', 'hint', 'level', 'step_criteria', 'step_score', 'competence') in json format.
"""

templates["insert_problem"] = """
You are a sql database Chatbot.
For each given data, we want to make a sql query to insert data into the knowledge_map_db.problem table.
make the insert query for the knowledge_map_db.problem table.
Exemplary data format is as follows:
problem_id: serial (constraint: primary_key, you don't need to insert this column)
question: text
solution: text
hint: text
level: int
step_criteria: json
step_score: 'int[]'
competence: 'numeric[]'
sub_unit_id: int

Please answer insert query for the knowledge_map_db.problem table without any other message.
Also, Do not use ```sql{answer}``` format , just print text.
"""

templates["insert_knowledge_problem"] = """
You are a sql database Chatbot.
For given data, we want to make a sql query to insert data into the knowledge_map_db.knowledge_problem which shows the relationship between knowledge and problem.

Exemplary data format is as follows:
knowledge_problem table:
knowledge_id: int 
problem_id: int (we don't know )

Based on given data, make the insert query for the knowledge_map_db.knowledge_problem table without any other message.
Do not use ```sql{answer}``` format , just print text.
Template :
WITH inserted_problem AS (
    SELECT problem_id
    FROM knowledge_map_db.problem
    WHERE question = '{{question}}'
)
INSERT INTO knowledge_map_db.knowledge_problem (knowledge_id, problem_id)
SELECT knowledge_id, ip.problem_id
FROM inserted_problem ip, (VALUES ({{knowledge_id}}), ({{knowledge_id}}), ({knowledge_id})) AS knowledge(knowledge_id);
"""


In [7]:
# Connect to the SQL database and get the knowledge and sub_unit tables as dataframes
connection = init_connection()

# Initialize the chat models
gpt_4 = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.3, api_key=st.secrets["apikey"])
gpt_3 = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0.3, api_key=st.secrets["apikey"])

# Define the function to run a single chat interaction with the given chat model with prompt
def langchain_single_chat(chat, system_message, human_message):
  """
  Run a single chat interaction with the given chat model.

  Args:
  chat (ChatOpenAI): The chat model.
  system_message (SystemMessage): The system message to send to the chat model.
  human_message (HumanMessage): The human message to send to the chat model.

  Returns:
  response (AIMessage): The response from the chat model.
  """
  # define the messages
  messages = [SystemMessage(content=system_message), HumanMessage(content=human_message)]
  # get the response from the chat model
  response = chat(messages)
  # return the response as a string
  return response.content

# get all the files in the nested directory ./MATH/
files = glob.glob("./MATH/" + "**/*.json", recursive=True)
# Shuffle the files
random.shuffle(files)

# Load a json file and print
for file_path in files[:1]:
  with open(file_path) as f:
    raw_data = json.load(f)
    raw_data = str(raw_data)
    print(raw_data)
    isHighSchoolLevel = langchain_single_chat(gpt_4, templates["give problem"], raw_data)
    print("isHighSchoolLevel:", isHighSchoolLevel)
    # Print the response content
    if isHighSchoolLevel == "1":
      relatedKnolwedge = langchain_single_chat(gpt_4, templates["knowledge_mapping"], raw_data)
      print("response:", relatedKnolwedge)
      relatedUnit = langchain_single_chat(gpt_4, templates["unit_mapping"], raw_data)
      print("response:", relatedUnit)
      postprocess_response = langchain_single_chat(gpt_4, templates["postprocess"], raw_data)
      print("postprocessed data:", postprocess_response)
      insert_problem_query = langchain_single_chat(gpt_3, templates["insert_problem"], f"{postprocess_response}, {relatedUnit}")
      print("insert_problem_query:", insert_problem_query)
      insert_knowledge_problem_query = langchain_single_chat(gpt_3, templates["insert_knowledge_problem"], f"{postprocess_response}, {relatedKnolwedge}")
      print("insert_knowledge_problem_query:", insert_knowledge_problem_query)
      # Conduct insert query on the database
      cursor.execute(insert_problem_query)
      cursor.execute(insert_knowledge_problem_query)
      cursor.close()
      connection.commit()

{'problem': 'Four people can paint a house in six hours. How many hours would it take three people to paint the same house, assuming everyone works at the same rate?', 'level': 'Level 2', 'type': 'Algebra', 'solution': 'The number of people painting the house and the amount of time it takes are inversely proportional. This means that if we let $n$ be the number of people, and $t$ be the time taken, the product $nt$ is a constant. Since 4 people can paint the house in 6 hours, $nt=(4)(6)=24$. Therefore, if three people were painting the same house, $nt=3t=24$, and $t=\\boxed{8}$.'}
isHighSchoolLevel: 1
response: ```json
[
  {"knowledge_id": 80, "knowledge_name": "명제"},
  {"knowledge_id": 147, "knowledge_name": "수열의 정의"},
  {"knowledge_id": 183, "knowledge_name": "방정식과 부등식에의 활용"}
]
```
response: ```json
{
  "sub_unit_id": 60,
  "sub_unit_name": "연립방정식"
}
```
postprocessed data: ```json
{
  "question": "네 사람이 집을 칠하는데 6시간이 걸립니다. 모든 사람이 같은 속도로 작업한다고 가정할 때, 세 사람이 같은 집을 칠하는데 걸리는 시간은 얼마입니까?",


### Task 2. SQL database chain

In [16]:
# Ref
# https://github.com/langchain-ai/langchain/issues/6918
# Talk to your Database using RAG and LLMs
# (ref: https://medium.com/@shivansh.kaushik/talk-to-your-database-using-rag-and-llms-42eb852d2a3c)

# Status: In progress. It just works for a single question not for retrival questions.
# To Do:
# 1. Add memory to the SQLDatabaseChain object.
# 2. Add some examples SQL queries to the memory.
# 3. Test the pgvector extension and similarity search.

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.3, api_key=st.secrets["apikey"])

# Initialize the SQL database connection
db, pg_uri = init_db()

# Create the SQLDatabaseChain object
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, top_k=3)

# Define the question to run the SQL query
question = "Find the all schemes in our db."

PROMPT = """ 
Given an input question, first create a syntactically correct postgresql query to run,  
then look at the results of the query and return the answer.  
The question: {question}
""".format(question=question)
# Run the SQL query using the db_chain object
db_chain.run(PROMPT)

/Users/yun/miniforge3/envs/brain/lib/python3.8/site-packages/langchain_experimental/sql/base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new SQLDatabaseChain chain...
 
Given an input question, first create a syntactically correct postgresql query to run,  
then look at the results of the query and return the answer.  
The question: Find the all schemes in our db.

SQLQuery:SQLQuery: SELECT DISTINCT table_schema FROM information_schema.tables WHERE table_schema NOT IN ('pg_catalog', 'information_schema') ORDER BY table_schema LIMIT 3;

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "SQLQuery"
LINE 1: SQLQuery: SELECT DISTINCT table_schema FROM information_sche...
        ^

[SQL: SQLQuery: SELECT DISTINCT table_schema FROM information_schema.tables WHERE table_schema NOT IN ('pg_catalog', 'information_schema') ORDER BY table_schema LIMIT 3;]
(Background on this error at: https://sqlalche.me/e/20/f405)

### Task 3. Chat with Memory

In [22]:
# To DO:
# 1. Test SQL memory in https://python.langchain.com/docs/integrations/memory/sql_chat_message_history
# 2. Save conversation history in the SQL database.
# 3. Save and load memory from the SQL database.
# 4. Test the connection with SQLdatabaseChain and openai assistant api.
# 5. Make a chatbot in UI using streamlit.
# 6. Test the cache in langchain.

from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.memory import PostgresChatMessageHistory
from sqlalchemy import create_engine
# engine = create_engine(pg_uri)
# set_llm_cache(SQLAlchemyCache(engine))

# LLM
llm = ChatOpenAI()

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Hello! How can I assist you today?'}

In [24]:
conversation({"question": "Can you save previous messages?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: Translate this sentence from English to French: I love programming.
AI: Sure! The translation of "I love programming" from English to French is "J'adore programmer."
Human: Can you save previous messages?

> Finished chain.


{'question': 'Can you save previous messages?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='Translate this sentence from English to French: I love programming.'),
  AIMessage(content='Sure! The translation of "I love programming" from English to French is "J\'adore programmer."'),
  HumanMessage(content='Can you save previous messages?'),
  AIMessage(content="As an AI language model, I don't have the capability to save or recall previous messages. Each interaction is treated as a separate query. Is there something specific you would like to refer back to?")],
 'text': "As an AI language model, I don't have the capability to save or recall previous messages. Each interaction is treated as a separate query. Is there something specific you would like to refer back to?"}

In [27]:
# Test the cache in langchain

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLAlchemyCache
from sqlalchemy import create_engine

cache_type = 'sqlalchemy' # or 'InMemoryCache'
if cache_type == 'sqlalchemy':
    engine = create_engine(pg_uri)
    set_llm_cache(SQLAlchemyCache(engine))
elif cache_type == 'InMemoryCache':
    set_llm_cache(InMemoryCache())

OperationalError: (psycopg2.OperationalError) connection to server at "147.47.200.145", port 34543 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Task 4. use Openai assistant api in Langchain

In [ ]:
# Ref1: https://python.langchain.com/docs/modules/agents/agent_types/openai_assistants
# Ref2: https://github.com/langchain-ai/langchain/blob/master/cookbook/openai_v1_cookbook.ipynb
# Ref3: https://www.youtube.com/watch?v=IZGBshGqB3g&t=1s
# Status: In progress. It could be used for the math assistant chatbot and auto-grading system.
# To Do:
# 1. GPT-4 vision test


# This is a test code for langchain with OpenAI Assistant api
openai.api_key = st.secrets["apikey"]

interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})